In [23]:
#Import the libraries I'll use.
import pandas as pd
import requests
from datetime import datetime
from bs4 import BeautifulSoup as bs
from igdb.wrapper import IGDBWrapper
import pickle
import bs4
import json
import re

# Created functions and did the scraping work here, and the finished functions are orginized below

# Working Area For Testing Stuff:

In [832]:
#Starting by looking at a specific game page and looking what class is the box info.
r = requests.get("https://en.wikipedia.org/wiki/Mortal_Kombat_II")
soups = bs(r.content,"lxml")

info_box = soups.find(class_="infobox hproduct")
info_rows = info_box.select("tr")
for row in info_rows:
    print(row.prettify())

<tr>
 <th class="infobox-above fn" colspan="2" style="font-size:125%;font-style:italic;">
  Mortal Kombat II
 </th>
</tr>

<tr>
 <td class="infobox-image" colspan="2">
  <a class="image" href="/wiki/File:Mortal_Kombat_II_boxart.png">
   <img alt="Mortal Kombat II boxart.png" data-file-height="366" data-file-width="272" decoding="async" height="296" src="//upload.wikimedia.org/wikipedia/en/thumb/d/df/Mortal_Kombat_II_boxart.png/220px-Mortal_Kombat_II_boxart.png" srcset="//upload.wikimedia.org/wikipedia/en/d/df/Mortal_Kombat_II_boxart.png 1.5x" width="220"/>
  </a>
  <div class="infobox-caption" style="font-size: 95%;">
   Cover artwork for the home versions
  </div>
 </td>
</tr>

<tr>
 <th class="infobox-label" scope="row" style="white-space:nowrap;padding-right:0.65em;">
  <a href="/wiki/Video_game_developer" title="Video game developer">
   Developer(s)
  </a>
 </th>
 <td class="infobox-data">
  <a href="/wiki/Midway_Games" title="Midway Games">
   Midway
  </a>
  <br/>
  <a class="mw

In [635]:
def remove_tags(soup):
    for tag in soup.find_all("sup"):
        tag.decompose() #This remove all the "sup" elements.

In [1099]:
#Creating a function to check for specific conditions in the another function to make the data values
#clean as possible.
def fixing_elems(elem):
    try:
        if elem.select("div > ul > li > a"):
            return [el.get_text() for el in elem.select("div > ul > li > a")] #This is for some "a" elements that didn't render through the web scraping.
        elif elem.select("div > ul > li > i"):
            return elem.get_text(" ",strip=True).replace("\xa0","").replace("\n","") #This return "i" elements that didn't render through the web scraping.
        elif elem.select("div li span"):
            elem = [el.next_sibling.strip(" )(") for el in elem.select("div li span")] 
            filter_obj = filter(lambda x: x != "", elem)
            without_empty_strings = list(filter_obj)
            return without_empty_strings #This returns some dates that are stored in a hidden list, and had to open them to get information I needed, and filter out empty string along the way.
        elif elem.find("#mw-content-text > div.mw-parser-output > table > tbody > tr > td"):
            return str(elem.get_text()).split(", ") #This splits long string that are at the same row.
        elif len(elem.find_all("a"))==1:
            return elem.get_text(" ",strip=True) #this checks if an "a" element in each key is single - then to return it not in a form of a list like the other conditions.
        elif elem.select("#mw-content-text > div.mw-parser-output > table.infobox.hproduct > tbody > tr > td > div > div > div"):
            return [el.get_text() for el in elem.select("#mw-content-text > div.mw-parser-output > table.infobox.hproduct > tbody > tr > td > div > div > div")] #This Checks for a specific value locked in a nested div and take the text out.
        elif elem.find("ul",{"class":'mw-collapsible-content hlist'}):
            return elem.a.get_text() #This checks for specific text that is inside a div of a div with no class or id.
        elif elem.select("div.plainlist ul li div.plainlist ul li"):
            elem = [elem.get_text() for elem in elem.select("div.plainlist ul li div.plainlist ul li",limit=3)]
            return elem #This checks for specific text that is above an expandeble list button and take only it.
        elif elem.find("li"):
            return [li.get_text().replace("\xa0","").replace("\n","").strip() for li in elem.find_all("li")] #This create lists and clean them from the "li" elements.
        elif elem.select("#mw-content-text > div.mw-parser-output > table > tbody > tr > td > a"):
            return [text.get_text() for text in elem.select("#mw-content-text > div.mw-parser-output > table > tbody > tr > td > a")] #This split "a" elements that became a long string.
        elif elem.find("b",{"Home versions":""}):
            elem = str(elem)
            elem = elem.replace("Home versions","")
            elem = bs(elem)
            elem = elem.get_text(" ",strip=True).replace("\xa0","").replace("\n","")
            return elem #This delete a specific section of "Home Version" games releases, since I decided to only use the first releases of the games.
        elif elem.find("br"):
            elem = str(elem)
            elem = elem.replace("<br>" , "").replace("\n","").strip()
            elem = bs(elem)
            elem = elem.get_text(", ",strip=True).replace("\xa0","").replace("\n","").split(", ")
            return elem #This deletes line breaks and replace it with a space.
        elif elem:
             return str(elem.get_text().replace("and ","")).split(", ") #This split some long strings and remove the word "and" and split them.
        else:
            return elem.get_text(" ",strip=True).replace("\xa0","").replace("\n","").strip() #This return everything else that is fine.
 
    except:
        if elem.select("div > div > div"):
            return [el.get_text(" ",strip=True).replace("\xa0","").replace("\n","") for el in elem.select("div > div > div")]
        elif elem.find(class_="mw-collapsible mw-collapsed"):  
            pass #This Check for the specific class (mostly of lists I don't need because they are an addition to the information) and skip it.
        elif elem.select("div > ul > li"):
            return [el.get_text() for el in elem.select("div > ul > li")] #This returns "ul" elements that are separated and combine them together in a list.

In [1111]:
#Creating a function that takes a URL & and getting the data by each page link and create dictionaries,
#sorted by the the categories as keys and the values as the keys information.
def get_info_box(url):
    r = requests.get(url)
    soups = bs(r.content,"lxml")
    info_box = soups.find(class_="infobox hproduct")
    info_rows = info_box.find_all("tr")
    
    remove_tags(soups)
    
    games_info = {}
    for index, row in enumerate(info_rows):
        if index==0:
            games_info["Title"] = row.find("th").get_text(" ",strip=True)
        elif index == 1: # That is the picture, I won't use it.
            continue 
        else:
            key = row.find("th").get_text()
            value = fixing_elems(row.find("td"))
            games_info[key] = value
    
    return games_info

{'Title': 'Donkey Kong',
 'Developer(s)': ['Nintendo R&D1', 'Ikegami Tsushinki'],
 'Publisher(s)': 'Nintendo',
 'Director(s)': 'Shigeru Miyamoto',
 'Producer(s)': 'Gunpei Yokoi',
 'Designer(s)': ['Shigeru Miyamoto', 'Gunpei Yokoi'],
 'Artist(s)': ['Shigeru Miyamoto'],
 'Composer(s)': ['Yukio Kaneoka'],
 'Series': 'Donkey Kong Mario',
 'Platform(s)': ['Arcade',
  'Game & Watch',
  'Atari 2600',
  'Intellivision',
  'ColecoVision',
  'Coleco Mini-arcade',
  'Atari 8-bit',
  'Famicom/NES',
  'TI-99/4a',
  'IBM PC',
  'Commodore 64',
  'VIC-20',
  'MSX',
  'ZX Spectrum',
  'Amstrad CPC',
  'Atari 7800',
  'Apple II',
  'Nintendo e-Reader',
  'Game Boy Advance'],
 'Release': ['mid-July, 1981',
  'July 31, 1981',
  'November 1981',
  'June 3, 1982',
  'July 1982',
  'August 1982',
  'August 1982',
  'August 1982',
  'June 1, 1983',
  'July 15, 1983',
  'June 1, 1986',
  'October 15, 1986',
  '1983',
  '1983',
  '1983',
  '1983',
  '1986',
  '1986',
  '1986',
  '1986',
  'April 8, 1988',
  '1

# Finish Project Area:

In [1112]:
def remove_tags(soup):
    for tag in soup.find_all("sup"):
        tag.decompose() #This removes all the "sup" elements.

In [1113]:
#Creating a function to check for specific conditions in the another function to make the data values
#clean as possible.
def fixing_elems(elem):
    try:
        if elem.select("div > ul > li > a"):
            return [el.get_text() for el in elem.select("div > ul > li > a")] #This is for some "a" elements that didn't render through the web scraping.
        elif elem.select("div > ul > li > i"):
            return elem.get_text(" ",strip=True).replace("\xa0","").replace("\n","") #This return "i" elements that didn't render through the web scraping.
        elif elem.select("div li span"):
            elem = [el.next_sibling.strip(" )(") for el in elem.select("div li span")] 
            filter_obj = filter(lambda x: x != "", elem)
            without_empty_strings = list(filter_obj)
            return without_empty_strings #This returns some dates that are stored in a hidden list, and had to open them to get information I needed, and filter out empty string along the way.
        elif elem.find("#mw-content-text > div.mw-parser-output > table > tbody > tr > td"):
            return str(elem.get_text()).split(", ") #This splits long string that are at the same row.
        elif len(elem.find_all("a"))==1:
            return elem.get_text(" ",strip=True) #this checks if an "a" element in each key is single - then to return it not in a form of a list like the other conditions.
        elif elem.select("#mw-content-text > div.mw-parser-output > table.infobox.hproduct > tbody > tr > td > div > div > div"):
            return [el.get_text() for el in elem.select("#mw-content-text > div.mw-parser-output > table.infobox.hproduct > tbody > tr > td > div > div > div")] #This Checks for a specific value locked in a nested div and take the text out.
        elif elem.find("ul",{"class":'mw-collapsible-content hlist'}):
            return elem.a.get_text() #This checks for specific text that is inside a div of a div with no class or id.
        elif elem.select("div.plainlist ul li div.plainlist ul li"):
            elem = [elem.get_text() for elem in elem.select("div.plainlist ul li div.plainlist ul li",limit=3)]
            return elem #This checks for specific text that is above an expandeble list button and take only it.
        elif elem.find("li"):
            return [li.get_text().replace("\xa0","").replace("\n","").strip() for li in elem.find_all("li")] #This create lists and clean them from the "li" elements.
        elif elem.select("#mw-content-text > div.mw-parser-output > table > tbody > tr > td > a"):
            return [text.get_text() for text in elem.select("#mw-content-text > div.mw-parser-output > table > tbody > tr > td > a")] #This split "a" elements that became a long string.
        elif elem.find("b",{"Home versions":""}):
            elem = str(elem)
            elem = elem.replace("Home versions","")
            elem = bs(elem)
            elem = elem.get_text(" ",strip=True).replace("\xa0","").replace("\n","")
            return elem #This delete a specific section of "Home Version" games releases, since I decided to only use the first releases of the games.
        elif elem.find("br"):
            elem = str(elem)
            elem = elem.replace("<br>" , "").replace("\n","").strip()
            elem = bs(elem)
            elem = elem.get_text(", ",strip=True).replace("\xa0","").replace("\n","").split(", ")
            return elem #This deletes line breaks and replace it with a space.
        elif elem:
             return str(elem.get_text().replace("and ","")).split(", ") #This split some long strings and remove the word "and" and split them.
        else:
            return elem.get_text(" ",strip=True).replace("\xa0","").replace("\n","").strip() #This return everything else that is fine.
 
    except:
        if elem.select("div > div > div"):
            return [el.get_text(" ",strip=True).replace("\xa0","").replace("\n","") for el in elem.select("div > div > div")] #Return texts that are under divs and return them in a list.
        elif elem.find(class_="mw-collapsible mw-collapsed"):  
            pass #This Check for the specific class (mostly of lists I don't need because they are an addition to the information) and skip it.
        elif elem.select("div > ul > li"):
            return [el.get_text() for el in elem.select("div > ul > li")] #This returns "ul" elements that are separated and combine them together in a list.

In [1114]:
#Creating a function that takes a URL & and getting the data by each page link and create dictionaries,
#sorted by the the categories as keys and the values as the keys information.
def get_info_box(url):
    r = requests.get(url)
    soups = bs(r.content,"lxml")
    info_box = soups.find(class_="infobox hproduct")
    info_rows = info_box.find_all("tr")
    
    remove_tags(soups)
    
    games_info = {}
    for index, row in enumerate(info_rows):
        if index==0:
            games_info["Title"] = row.find("th").get_text(" ",strip=True)
        elif index == 1: # That is the picture, I won't use it.
            continue 
        else:
            key = row.find("th").get_text()
            value = fixing_elems(row.find("td"))
            games_info[key] = value
    
    return games_info

In [1115]:
#Loaded the page I'm going to get all the video games list and iterate on each game to get the data
#with the help of the two functions i created above.
r = requests.get("https://en.wikipedia.org/wiki/List_of_video_games_considered_the_best")
soup_games = bs(r.content,"lxml")
games = soup_games.select(".wikitable.sortable i a")

wiki_path = "https://en.wikipedia.org"

game_info_list = []
for index, game in enumerate(games):
    try:
        relative_path = game["href"]
        full_wiki_path = wiki_path+relative_path
        fixing_elems(soup_games)
        title = game["title"]
        game_info_list.append(get_info_box(full_wiki_path))
        
    except Exception as e: #3 games won't open so I decided to pass them.
        print(game.get_text())
        print(e)

In [1116]:
#Cheking the length of the list I created and saw that it has the same length
#without the games number I couldn't iterate.
len(game_info_list)

236

In [1117]:
#Created a function to save my list as a json file, since it's easier to save dictionaries
#in this file type.
def save(title, data):
    with open(title, "w", encoding="utf8") as f:
        json.dump(data, f, ensure_ascii=False, indent=2)
        
save("Video Games Data Cleaned.json", game_info_list)

In [2]:
#Created a function to load my json file.
def load(title):
    with open(title, "r", encoding="utf8") as f:
        return json.load(f)
    
game_info_list = load("Video Games Data Cleaned.json")

## Functions To Parse The Dates With The patterns For Each Case:

In [1119]:
m_d_y = r"\w+\s\d+(,)\s\d{4}" #Month Day, Year. 
d_m_y = r"\d+\s\w+\s\d{4}" #Day, Month, Year.
m_y = r"\w+\s\d{4}" #Month and Year.
m_y_2 =r"^\w+\d{4}" #Another Month and Year format.
y = r"\d{4}" #Year only.
datetime_fmt = r"[(\s?]\d{4}(-)\d{2}(-)\d{2}[\s?)]" #Dates that had a datetime format in the page itself.

#Each Function name represent the regex pattern it supposed to find.
def datetime_fmt_version(date):
    value = datetime.strptime(date,'(%Y-%m-%d)')
    value = value.strftime('%Y-%m-%d')
    return value

def m_d_y_version(date):
    value = datetime.strptime(date,'%B %d, %Y')
    value = value.strftime('%Y-%m-%d')
    return value

def d_m_y_version(date):
    value = datetime.strptime(date,'%d %B %Y')
    value = value.strftime('%Y-%m-%d')
    return value

def m_y_version(date):
    value = datetime.strptime(date,'%B %Y')
    value = value.strftime('%Y-%m')
    return value

def m_y_2_version(date):
    value = datetime.strptime(date,'%B%Y')
    value = value.strftime('%Y-%m')
    return value

def y_version(date):
    value = datetime.strptime(date,'%Y')
    value = value.strftime('%Y')
    return value

## Function That tries To Parse The Dates With The Function Above, If They're Stored In A List:

In [1120]:
def dates_in_list(date):
    try:
        if re.search(m_d_y,date[0]):
            value = re.search(m_d_y,date[0]).group()
            return m_d_y_version(value) #If the first argument is valid, it will parse it into a datetime object.
    except:
        pass 
    try:
        if re.search(d_m_y,date[0]):
            value = re.search(d_m_y,date[0]).group()
            return d_m_y_version(value) #If the first argument is valid, it will parse it into a datetime object.
    except:
        pass
    try:
        if re.search(m_d_y,date[0])==None:
            value = re.search(m_d_y,date[1]).group()
            return m_d_y_version(value) #If the first argument is not valid, it will parse the second element from the list, and parse it into a datetime object.
    except:
        pass
    try:
        if re.search(d_m_y,date[0])==None:
            value = re.search(d_m_y,date[1]).group()
            return d_m_y_version(value) #If the first argument is not valid, it will parse the second element from the list, and parse it into a datetime object.
    except:
        pass
    try:
        if re.search(m_y,date[0]):
            value = re.search(m_y,date[0]).group()
            return m_y_version(value) #If the first argument is valid, it will parse it into a datetime object.
    except:
        pass
    try:
        if re.search(m_y,date[0]):
            value = re.search(m_y,date[1]).group()
            return m_y_version(value) #If the first argument is not valid, it will parse the second element from the list, and parse it into a datetime object.
    except:
        pass
    try:
        if re.search(m_y_2,date[0]):
            value = re.search(m_y_2,date[0]).group()
            return m_y_2_version(value) #If the first argument is valid, it will parse it into a datetime object.
    except:
        pass
    try:
        if re.search(m_y_2,date[0]):
            value = re.search(m_y_2,date[1]).group()
            return m_y_2_version(value) #If the first argument is not valid, it will parse the second element from the list, and parse it into a datetime object.
    except:
        pass
    try:
        if re.search(datetime_fmt,date[0]):
            value = re.search(datetime_fmt,date[0]).group()
            return datetime_fmt_version(value) #If the first argument is valid, it will parse it into a datetime object.
    except:
        pass
    try:
        if re.search(datetime_fmt,date[0]):
            value = re.search(datetime_fmt,date[1]).group()
            return datetime_fmt_version(value) #If the first argument is not valid, it will parse the second element from the list, and parse it into a datetime object.
    except:
        pass
    try:
        if re.search(y,date[0]):
            value = re.search(y,date[0]).group()
            return y_version(value) #If the first argument is valid, it will parse it into a datetime object.
    except:
        pass
    try:
        if re.search(y,date[0]):
            value = re.search(y,date[1]).group()
            return y_version(value) #If the first argument is not valid, it will parse the second element from the list, and parse it into a datetime object.
    except:
        pass
    try:
        if len(date)==2:
            value = ", ".join(date)
            return m_d_y_version(value) #If the list is built from a separate date that has two elements only, it'll combine them and parse it into a datetime object.
        elif len(date)>2:
            date = date[:2]
            value = ", ".join(date)
            value = value.split(":")
            value = value[1].strip()
            return m_d_y_version(value) #If the list is built from more then 2 elements, it'll combine them and parse the first date into a datetime object.
    except:
        pass

In [1121]:
#The main function that combines all the functions above.
def date_conversion(date):
    if isinstance(date,list):
        return dates_in_list(date) #If the date is in a list, it will use the "dates_in_list" function.
    elif re.search(m_d_y,date):
        value = re.search(m_d_y,date).group()
        return m_d_y_version(value) #If the pattern is a string, check the pattern of Month Day, Year format
    elif re.search(d_m_y,date):
        value = re.search(d_m_y,date).group()
        return d_m_y_version(value) #If the pattern is a string, check the pattern of Day Month Year format.
    elif re.match(m_y,date):
        value = re.search(m_y,date).group()
        return m_y_version(value) #If the pattern is a string, check the pattern of Month Year format.
    elif re.search(m_y_2,date):
        value = re.search(m_y_2,date).group()
        return m_y_2_version(value) #If the pattern is a string, check the pattern of MonthYear format (with no space between them).
    elif re.search(datetime_fmt,date):
        value = re.search(datetime_fmt,date).group()
        return datetime_fmt_version(value) #If the pattern is a string, check the pattern of Year-Month-day.
    elif re.search(y,date):
        value = re.search(y,date).group()
        return y_version(value) #If the pattern is a string, check the pattern of Year. 

In [1128]:
#Apply the functions I wrote above to create a new key for each dictionary with the datetime object.
for index, game in enumerate(game_info_list):
    game["Release_datetime_obj"] = date_conversion(game.get("Release","N/A"))

In [17]:
#A function to save the new cleaned more data with the dates as datetime objects using pickle.
def save_pkl_file(title, data):
    with open(title, "wb") as file:
        pickle.dump(data,file)
        
save_pkl_file("Video Games Data With Datetime.pickle",game_info_list)

In [18]:
#A function to load the clean data with the dates as datetime objects.
def load_pkl_file(file):
    with open("Video Games Data With Datetime.pickle", "rb") as file:
        return pickle.load(file)

game_info_list = load_pkl_file("Video Games Data With Datetime.pickle")

In [52]:
#Load the saved pickle file before.
game_info_list = load_pkl_file("Video Games Data With Datetime")

In [83]:
#Save the list of game dictionaries as a dataframe.
df = pd.DataFrame(game_info_list)
df

,Title,Developer(s),Publisher(s),Series,Platform(s),Release,Genre(s),Mode(s),Release_datetime_obj,Designer(s),Engine,Arcade system,Programmer(s),Composer(s),Director(s),Producer(s),Artist(s),Writer(s)
0,The Oregon Trail,"[Don Rawitsch, Bill Heinemann, Paul Dillenberger]",MECC,The Oregon Trail,"[Mainframe, HP 2100, CDC Cyber 70/73-26, PC, A...","[December 3, 1971, 1975]",Strategy,Single-player,1971-12-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Pong,Atari,[Namco],NaN,"[Arcade, dedicated consoles]","[29 November 1972, November 1973, 1973]",Sports,"[Single-player, multiplayer]",1972-11-29,Allan Alcorn,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Zork,Infocom,"[Personal Software, Activision]",NaN,"[PDP-10, Atari 8-bit, C64, CP/M, TRS-80, IBM P...","[1977 (PDP-10), 1980 (, Zork I, ), 1981 (, Zor...",Text adventure,Single-player,1977,"[Tim Anderson, Marc Blank, Dave Lebling, Bruce...",ZIL,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Space Invaders,Taito,[Midway],Space Invaders,"[Arcade, Atari 2600, Atari 5200, Atari 8-bit, ...","[April 1, 1978, November 1978, 1978, March 10,...",Shoot 'em up,"Single-player , 2 players alternating",1978-04-01,Tomohiro Nishikado,NaN,Taito 8080,NaN,NaN,NaN,NaN,NaN,NaN
4,Asteroids,"Atari, Inc.","[Taito, Sega]",Asteroids,"[Arcade, Atari 2600, Atari 7800, Atari 8-bit, ...","[November 1979, 1979, January 1980, July 1981,...",Multidirectional shooter,"[Single-player, multiplayer]",1979-11,"[Lyle Rains, Ed Logg]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
231,Bloodborne,FromSoftware,Sony Computer Entertainment,NaN,PlayStation 4,"[March 24, 2015, March 25, 2015, March 26, 201...",Action role-playing,"[Single-player, multiplayer]",2015-03-24,[Kazuhiro Hamatani],NaN,NaN,[Jun Ito],"[Cris Velasco, Michael Wandmacher]",Hidetaka Miyazaki,"[Masaaki Yamagiwa, Teruyuki Toriyama]",[Ryo Fujimaki],NaN
232,The Witcher 3: Wild Hunt,CD Projekt Red,CD Projekt,The Witcher,"[Microsoft Windows, PlayStation 4, PlayStation...","[Windows, PS4, Xbox One, 19 May 2015, Nintendo...",Action role-playing,Single-player,2015-05-19,NaN,REDengine 3,NaN,NaN,[Marcin Przybyłowicz],"[Konrad Tomaszkiewicz, Mateusz Kanik, Sebastia...","[Piotr Krzywonosiuk, Jędrzej Mróz]",[Marian Chomiak],[Marcin Blacha]
233,Inside,Playdead,[Playdead],NaN,"[Xbox One, Microsoft Windows, PlayStation 4, N...","[Xbox One, June 29, 2016, Microsoft Windows, J...","[Puzzle-platformer, adventure]",Single-player,2016-06-29,NaN,Unity,NaN,NaN,[Martin Stig Andersen],[Arnt Jensen],"[Arnt Jensen, Dino Patti]",NaN,NaN
234,Overwatch,Blizzard Entertainment,[Blizzard Entertainment],NaN,"[Microsoft Windows, PlayStation 4, Xbox One, N...","[Windows, PS4, Xbox One, May 24, 2016, Nintend...",First-person shooter,Multiplayer,2016-05-24,"[Jeremy Craig, Michael Elliott, Scott Mercer]",NaN,NaN,"[Mike Elliott, John LeFleur]",[Derek Duke],"[Jeff Kaplan, Chris Metzen]",NaN,"[William Petras, Arnold Tsang]",[Alyssa Wong]


In [84]:
#Remove the release column and use the datetime column for future analysis.
df = df.set_index("Release_datetime_obj").drop("Release",axis=1)

In [79]:
#Created a dictionary to fill NaN values that weren't published.
fill_na = {}
for col in df.columns:
    fill_na[col] = f"{col} Not Available"

In [85]:
#Fill the NaN values with the dictionary above.
df.fillna(fill_na,inplace=True)

In [87]:
#Save the data as a csv file.
df.to_csv("Video Games Dataset.csv")

# Continue The Analysis On postgrSQL :)